In [4]:
# import basic modules here 
import numpy as np
import cv2
import os 
import glob
import time
import sys
import json
import slidingwindow as sw
import matplotlib.pyplot as plt 
import pycocotools.mask as maskUtils
import mmcv 
import torch
from pycocotools.coco import COCO
from pycocotools import coco
from IPython.display import clear_output, display

# import mmdet-related modules 
from mmdet.apis import init_detector, inference_detector
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from scipy.ndimage import binary_fill_holes

# import shm_tools here
sys.path.append('..')
from shm_tools.shm_utils import imread, imwrite, inference_detector_sliding_window, AverageMeter, comparison_operator, intersectionAndUnion


In [9]:
path = '/home/byunghyun/mmdetection/data/samples_road'
im_list = glob.glob(os.path.join(path,'*.jpg')) + glob.glob(os.path.join(path,'*.JPG'))

config = '/home/byunghyun/mmdetection/project_work_dirs/express_cor/configs/cascade_mask_rcnn_x101_64x4d_fpn_carafe_1x_road_crack_200318.py'
checkpoint = '/home/byunghyun/mmdetection/project_work_dirs/express_cor/work_dirs/cascade_mask_rcnn_x101_64x4d_fpn_carafe_1x_road_crack_200318/epoch_30.pth'
device = 'cuda:0'
model = init_detector(config, checkpoint, device=device)

intersection_meter = AverageMeter()
union_meter = AverageMeter()
target_meter = AverageMeter()

intersection_list = []
union_list = []
target_list = []

color_mask = np.array([255, 0, 0], dtype=np.uint8)

for img_num, img_path in enumerate(im_list): 
    clear_output(wait=True)
    print('Image No. '+ str(img_num+1) + 'is under process')
    
    img_name = os.path.basename(img_path)   
    
    img_result, mask_output = inference_detector_sliding_window(model, img_path, color_mask, score_thr = 0.3, window_size = 1024, overlap_ratio = 0)
    
    num_class = 5 # background + object classes 
    
    
    seg_sum = np.zeros((img_result.shape[0], img_result.shape[1], num_class), dtype=np.float32)
    seg_sum[:, :, 1] = mask_output
    
    seg_2d = np.zeros((img_result.shape[0], img_result.shape[1]), dtype = np.uint8)
    seg_2d[seg_sum[:, :, 2] == 1] = 2 # efflorescence _non_overlap
    seg_2d[seg_sum[:, :, 4] == 1] = 4 # spalling 
    seg_2d[seg_sum[:, :, 3] == 1] = 3 # rebar exposure 
    seg_2d[seg_sum[:, :, 1] == 1] = 1 # crack 
    
    fast_inference = False
    annot_path = os.path.join(path, 'annot')
    annot_img_list = glob.glob(os.path.join(annot_path, os.path.splitext(img_name)[0]+ '*.jpg'))

    lower_thr = 150
    upper_thr = 200

    COLOR_OPTIONS = {
        'red'    :  [ '>', upper_thr,  '<', lower_thr, '<', lower_thr],
        'green'  : [ '<', lower_thr,  '>', upper_thr, '<', lower_thr],
    #     'blue'   :  [ '<', lower_thr,  '<', lower_thr, '>', upper_thr],
        'cyan'   : [ '<', 150,  '>', 200, '>', 200],
        'yellow' :  [ '>', 200,  '>', 200, '<', 150]
    }


    damage_areas = np.zeros((img_result.shape[0], img_result.shape[1], 4), dtype = np.uint8)

    for annot_img_path in annot_img_list:

        annot_img = imread(annot_img_path)

        for color_num, color in enumerate(COLOR_OPTIONS) : 

            rgb_thr = COLOR_OPTIONS[color]
            R = comparison_operator(annot_img[:, :, 0], rgb_thr[1], rgb_thr[0])
            G = comparison_operator(annot_img[:, :, 1], rgb_thr[3], rgb_thr[2])
            B = comparison_operator(annot_img[:, :, 2], rgb_thr[5], rgb_thr[4])

            damage_area = (R & G & B)
            damage_area = np.asarray(damage_area, dtype = np.uint8)
            damage_area = binary_fill_holes(damage_area).astype('uint8')
            damage_areas[:, :, color_num] = damage_areas[:, :, color_num] + damage_area

    damage_areas[damage_areas > 1 ] = 1

    seg_gt_2d = np.zeros((annot_img.shape[0], annot_img.shape[1]), dtype = np.uint8)

    seg_gt_2d[damage_areas[:, :, 3] == 1] = 4 # spalling 
    seg_gt_2d[damage_areas[:, :, 1] == 1] = 2 # efflorescence 
    seg_gt_2d[damage_areas[:, :, 2] == 1] = 3 # rebar exposure 
    seg_gt_2d[damage_areas[:, :, 0] == 1] = 1 # crack 


    intersection, union, target = intersectionAndUnion(seg_2d, seg_gt_2d, 5)
    intersection_list.append(intersection)
    union_list.append(union)
    target_list.append(target)
    intersection_meter.update(intersection[1:])
    union_meter.update(union[1:])
    target_meter.update(target[1:])
    accuracy = sum(intersection_meter.val) / (sum(target_meter.val) + 1e-10)

iou_class = intersection_meter.sum / (union_meter.sum + 1e-10)
accuracy_class = intersection_meter.sum / (target_meter.sum + 1e-10)
mIoU = np.mean(iou_class)
mAcc = np.mean(accuracy_class)
allAcc = sum(intersection_meter.sum) / (sum(target_meter.sum) + 1e-10)

print('mIoU:', mIoU, 'mAcc:', mAcc, 'allAcc:', allAcc)
    

Image No.1is under process


inference by sliding window on P-041-E1-미호천교(서울)_s000416000.jpg: 100%|##########| 190/190 [00:32<00:00,  5.89it/s]


mIoU: 0.024889670217562578 mAcc: 0.1918823075032471 allAcc: 0.7675292300129885
